The simulation is an agent-based model and is defined by the following classes:
- Simulation
- Driver
- Restaurant
- Customer
- Order

Simulation is the main agent that contains the city graph and global information pertaining to all actors within it. It is therefore only instantiated once.
The Driver, Restaurant & Customer agents are pre-defined as per the parameters, while the Order agents are defined dynamically as the simulation progresses. Of these, the Driver agent is the most complex as it needs to have decision-making capabilities with regards to picking/handing-over/delivering orders.

In a typical step without handoffs, the following events will occur:
- Each restaurant will receive a number of orders (following a Poisson Distribution) from random customers. 
- Upon receiving an order, the restaurant will emit a call (through the method 'allocate_to_driver') to assign the order to the closest available driver. If the driver does not have the capacity to take the order, he will refuse it and the next nearest driver will be called, and so on. If none of the drivers are able to take the order, it will be added to the restaurant's backlog & an attempt to allocate will be made in the next timestep (care is taken to ensure that older orders are given priority).
- After this, the driver has to decide where to move (method 'update_target'). At any time, the driver has 2 lists of orders in mind, called the 'pending_orders' (which the driver has accepted to pick but is not yet carrying) and 'carrying_orders' (which the driver has already picked). By default, the driver will always prioritize 'pending_orders' i.e. it will update its target location to the restaurant and identify the shortest path to get there. If the 'pending_orders' is empty, the driver will look at 'carrying_orders' and update its target to the customer' location. In both cases, the nearest restaurant/customer is selected. Finally, if both lists are empty, then the driver will return to its original location and await an order.
- Following this, the 'step' method is called, whereby the driver will move along the shortest path it has identified to the next target. Concretely, he will move along the current edge at the pre-defined speed. If the distance to the next node is higher than the speed, he will remain along the edge and will only advance as far as his speed allows. However, if the distance to the node is less, he will snap to the node and then checks to see where this node lies along the path. If it is merely a junction, he will update accordingly so that in the next step he moves along the next edge. However, if the node is a restaurant or customer, he will pickup or deliver the order and update his parameters.
- The cycle then repeats.

In a step with handoffs, the same set of events will occur, plus a check (method 'check_all_handoffs') to see if there is the opportunity for any driver to handover its order to a nearby driver who is headed in a similar direction. For this, 2 threshold parameters are defined ('handover_driver_threshold' & 'handover_customer_threshold') to ensure that they are close enough as for the handover to make sense. If there is a handover opportunity, the drivers positions will be fixed for the duration that it would take for them to travel the distance to each other & back (in order to physically handover the order). This is a reasonable simplification in order to avoid having to explicitly update the driver targets/steps, as this would add unnecessarily complexity. Once the handoff is complete & the drivers are free to move again, it will be observed that their orders have swapped and the driver who handed over is now free to accept a call from another restaurant rather than head to the customer.

It is clear from this that careful calibration of total driver carrying capacity vs. total order income rate (via the poisson distribution) is needed to ensure that all orders can be attended to in a timely manner. In a well-balanced system the inflow & outflow matched pretty well, but in some cases it was observed that an overwhelming backlog would buildup resulting in a slow iterations and very high delivery time.